In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# change directory to the root of the project
from os import chdir

chdir("../..")

In [4]:
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
from requests import get

# https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/330020_XXXX_PuntoRocio_

In [15]:
from os import remove


def get_dew_point(
    input_path: str,
    station_id: str,
) -> None:
    """_summary_

    Parameters
    ----------
    input_path : str
        _description_
    station_id : str
        _description_
    """
    # download and uncompress zip file
    print(f"Downloading dew point data for station {station_id}")
    zip_file_url = f"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/{station_id}_XXXX_PuntoRocio_"  # noqa: E501
    req = get(zip_file_url, stream=True).content

    zip_file = ZipFile(BytesIO(req))
    zip_info = zip_file.infolist()[0]
    zip_info.filename = "dew_point_history.csv"

    print(f"Uncompressing dew point data for station {station_id}")
    extract_path = f"{input_path}/stations/{station_id}"
    zip_file.extract(zip_info, path=extract_path)

    # replace ',' with ';' in .csv file to allow correct column separation
    history_path = f"{input_path}/stations/{station_id}/{zip_info.filename}"

    with open(history_path, "r") as f:
        lines = f.readlines()
        lines = map(lambda x: x.replace(",", ";"), lines)

    with open(history_path, "w") as f:
        f.writelines(lines)

    dew_point_data = pd.read_csv(f"{extract_path}/dew_point_history.csv", sep=";")

    dew_point_data.drop(columns=["CodigoNacional"], inplace=True)
    dew_point_data.rename(
        columns={
            "momento": "timestamp",
            "Td_Valor": "dew_point_temp",
        },
        inplace=True,
    )
    dew_point_data.set_index("timestamp", inplace=True)

    dew_point_data.to_parquet(f"{extract_path}/dew_point_history.parquet")
    remove(f"{extract_path}/dew_point_history.csv")


get_dew_point(
    "data/input",
    330020,
)

Uncompressing dew point data for station 330020


In [18]:
dew_point_data = pd.read_parquet("data/input/stations/330020/dew_point_history.parquet")
dew_point_data

,dew_point_temp
timestamp,
01-03-1967 00:00:00,NaN
01-03-1967 12:00:00,11.6
01-03-1967 18:00:00,16.0
02-03-1967 00:00:00,17.9
02-03-1967 12:00:00,12.7
...,...
05-10-2023 03:00:00,5.8
05-10-2023 04:00:00,6.2
05-10-2023 05:00:00,6.1
